In [1]:
import random
from PyQt5.QtWidgets import QApplication, QLabel, QPushButton, QWidget, QGridLayout, QHBoxLayout, QVBoxLayout, QLineEdit, QSlider
import pyqtgraph as pg
from PyQt5.QtCore import Qt, QTimer
import time
from pyqtgraph.Qt import QtCore
import sys
from tclab import clock, setup
import pandas as pd
import numpy as np


class Window(QWidget):
    def __init__(self):
        super().__init__()

        

        self.setWindowTitle('Anneal control')   
        layout = QVBoxLayout()
        upper_layout = QHBoxLayout()
#UPPER LEFT LAYOUT
        upper_left_layout = QGridLayout()
        PS_readout = QLabel('Power Supply Readout')
        PS_readout.setAlignment(Qt.AlignHCenter)
        #VOLTAGE
        PS_voltage_readout = QLabel('Voltage (V)')
        PS_voltage_readout.setAlignment(Qt.AlignHCenter)
        PS_voltage_readout_display = QHBoxLayout()
        PS_voltage_readout_display.setAlignment(Qt.AlignHCenter)
        self.PS_voltage_readout_display_number = QLineEdit(readOnly = True)
        self.PS_voltage_readout_display_number.setAlignment(Qt.AlignRight)
        PS_voltage_readout_display.addWidget(self.PS_voltage_readout_display_number)
        #CURRENT
        PS_current_readout = QLabel('Current (A)')
        PS_current_readout.setAlignment(Qt.AlignHCenter)
        PS_current_readout_display = QHBoxLayout()
        PS_current_readout_display.setAlignment(Qt.AlignHCenter)
        self.PS_current_readout_display_number = QLineEdit(readOnly = True) 
        self.PS_current_readout_display_number.setAlignment(Qt.AlignRight)

        PS_current_readout_display.addWidget(self.PS_current_readout_display_number)

        self.plot_P = pg.PlotWidget()
        self.curve_P=self.plot_P.plot(pen='y')
        self.plot_P.setLabel('left', "POWER", units='W')
        self.plot_P.setLabel('bottom', "Time", units='s')
        self.plot_P.setTitle('POWER')
        upper_left_layout.addWidget(PS_readout,1,1,1,2)
        upper_left_layout.addWidget(PS_voltage_readout,2,1)
        upper_left_layout.addLayout(PS_voltage_readout_display,3,1)
        upper_left_layout.addWidget(PS_current_readout,2,2)
        upper_left_layout.addLayout(PS_current_readout_display,3,2)
        upper_left_layout.addWidget(self.plot_P,4,1,1,2)

        upper_layout.addLayout(upper_left_layout)


#UPPER RIGHT LAYOUT
    #Buttons and displays
        upper_right_layout = QGridLayout()
        Pyro_readout = QLabel('Pyrometer readout')
        Pyro_readout.setAlignment(Qt.AlignHCenter)
        #TEMPERATURE
        Pyro_temp_readout = QLabel('Temperature (C)')
        Pyro_temp_readout.setAlignment(Qt.AlignHCenter)
        Pyro_temp_readout_display = QHBoxLayout()
        Pyro_temp_readout_display.setAlignment(Qt.AlignHCenter)
        self.Pyro_temp_readout_display_number = QLineEdit()#readOnly = True)
        self.Pyro_temp_readout_display_number.setAlignment(Qt.AlignRight)
        Pyro_temp_readout_display.addWidget(self.Pyro_temp_readout_display_number)

        
        #PRESSURE
        Pyro_pres_readout = QLabel('Pressure (mbar)')
        Pyro_pres_readout.setAlignment(Qt.AlignHCenter)
        Pyro_pres_readout_display = QHBoxLayout()
        Pyro_pres_readout_display.setAlignment(Qt.AlignHCenter)
        self.Pyro_pres_readout_display_number = QLineEdit('1.e-9')#readOnly = True)
        self.Pyro_pres_readout_display_number.setAlignment(Qt.AlignRight)        
        Pyro_pres_readout_display.addWidget(self.Pyro_pres_readout_display_number)
                                   

    #Plots   
        self.plot_T = pg.PlotWidget()
        self.curve_T = self.plot_T.plot(pen='y')
        self.plot_T.setLabel('left', "Temperature", units='C')
        self.plot_T.setLabel('bottom', "Time", units='s')
        self.plot_T.setTitle('TEMPERATURE')
    #Adding to the layout
        upper_left_layout.addWidget(Pyro_readout,1,3,1,2)
        upper_left_layout.addWidget(Pyro_temp_readout,2,3)
        upper_left_layout.addLayout(Pyro_temp_readout_display,3,3)
        upper_left_layout.addWidget(Pyro_pres_readout,2,4)
        upper_left_layout.addLayout(Pyro_pres_readout_display,3,4)
        upper_left_layout.addWidget(self.plot_T,4,3,1,2)

        upper_layout.addLayout(upper_right_layout)

        layout.addLayout(upper_layout)

# START/STOP BUTTONS
        bottom_layout = QHBoxLayout()
        self.startBtn = QPushButton('start')     #start button
        bottom_layout.addWidget(self.startBtn)
        self.stopBtn = QPushButton('stop')       #stop button
        bottom_layout.addWidget(self.stopBtn)
        layout.addLayout(bottom_layout)


# RAMP LAYOUT
    # Buttons 1st row
        Ramp_layout = QGridLayout()
        Temp_ramp = QLabel('Temperature ramp')
        Temp_ramp.setAlignment(Qt.AlignHCenter)
        Target_temp = QLabel('Target Temperature (C)')
        Target_temp.setAlignment(Qt.AlignHCenter)
        self.Target_temp_display = QLineEdit('70')
        self.Target_temp_display.setAlignment(Qt.AlignHCenter)
        Power_limit = QLabel('Power Limit (W)')
        Power_limit.setAlignment(Qt.AlignHCenter)
        self.Power_limit_display = QLineEdit('100')
        self.Power_limit_display.setAlignment(Qt.AlignHCenter)
        Press_limit = QLabel('Pressure Limit (mbar)')
        Press_limit.setAlignment(Qt.AlignHCenter)
        self.Press_limit_display = QLineEdit('1.0e-6')
        self.Press_limit_display.setAlignment(Qt.AlignHCenter)
        Power_knob = QLabel('Manual control')
        Power_knob.setAlignment(Qt.AlignHCenter)
        self.Power_knob = QSlider(Qt.Horizontal)
    # Adding to the layout
        Ramp_layout.addWidget(Temp_ramp,6,3,1,4)
        Ramp_layout.addWidget(Target_temp,7,3)
        Ramp_layout.addWidget(self.Target_temp_display,8,3)
    # Ramp_layout.addWidget(Target_temp_unit,8,4)
        Ramp_layout.addWidget(Power_limit,7,4)
        Ramp_layout.addWidget(self.Power_limit_display,8,4)
        Ramp_layout.addWidget(Press_limit,7,5)
        Ramp_layout.addWidget(self.Press_limit_display,8,5)
        Ramp_layout.addWidget(Power_knob,7,6)
        Ramp_layout.addWidget(self.Power_knob,8,6)
        self.Power_knob.setMinimum(0)
        self.Power_knob.setMaximum(100)
        self.Power_knob.valueChanged.connect(self.knobbing)
    # Buttons 2nd row
        Ramp_time = QLabel('Ramp time (mins)')
        Ramp_time.setAlignment(Qt.AlignHCenter)
        self.Ramp_time_display = QLineEdit('3')
        self.Ramp_time_display.setAlignment(Qt.AlignHCenter)
        Anneal_time = QLabel('Anneal time (mins)')
        Anneal_time.setAlignment(Qt.AlignHCenter)
        self.Anneal_time_display = QLineEdit('3')
        self.Anneal_time_display.setAlignment(Qt.AlignHCenter)
        Cooldown_time = QLabel('Cooldown time (mins)')
        Cooldown_time.setAlignment(Qt.AlignHCenter)
        self.Cooldown_time_display = QLineEdit('3')
        self.Cooldown_time_display.setAlignment(Qt.AlignHCenter)
        Wait_time = QLabel('Wait time (mins)')
        Wait_time.setAlignment(Qt.AlignHCenter)
        self.Wait_time_display = QLineEdit('1')
        self.Wait_time_display.setAlignment(Qt.AlignHCenter)
    # Adding to the layout
        Ramp_layout.addWidget(Ramp_time,9,3)
        Ramp_layout.addWidget(self.Ramp_time_display,10,3)
        Ramp_layout.addWidget(Anneal_time,9,4)
        Ramp_layout.addWidget(self.Anneal_time_display,10,4)
        Ramp_layout.addWidget(Cooldown_time,9,5)
        Ramp_layout.addWidget(self.Cooldown_time_display,10,5)
        Ramp_layout.addWidget(Wait_time,9,6)
        Ramp_layout.addWidget(self.Wait_time_display,10,6)
        layout.addLayout(Ramp_layout)
        self.RT_setpoint = 21
        self.current_setpoint = self.RT_setpoint
        Bottom_Btns_layout = QHBoxLayout()
        self.RampBtn = QPushButton('Anneal')     #start button
        self.StopAnnealBtn = QPushButton('STOP')
        self.RampDescr = QLabel('NO OPERATION')       #stop button
        self.RampDescr.setAlignment(Qt.AlignHCenter)
        self.RampStp = QLabel('stp:' + str(self.current_setpoint))       #stop button
        self.RampStp.setAlignment(Qt.AlignHCenter)
        Bottom_Btns_layout.addWidget(self.RampBtn)
        Bottom_Btns_layout.addWidget(self.StopAnnealBtn)
        Bottom_Btns_layout.addWidget(self.RampDescr)
        Bottom_Btns_layout.addWidget(self.RampStp)
        layout.addLayout(Bottom_Btns_layout)        
        
        
        
#SET LAYOUT
        self.setLayout(layout)
#ACTIVATE METHODS
        self.timer = QTimer()
        self.read_Data()
        self.Anneal()
        self.Create_PID()

        
    def record_file(self):
        self.df.iloc[[self.index]].to_csv('Anneal_Data.csv', mode='a', index = False, header= False)
        
    def knobbing(self):
        lab.U1 = self.Power_knob.value()
        self.current_setpoint = lab.T1

    def PID(self, Kp, Ki, Kd, MV_bar=0, beta=1, gamma=0):
    # initialize stored data
        eD_prev = 0
        t_prev = -1#00#00
        I = 0
        P = 0
        D = 0
    
    # initial control
        MV = MV_bar
    
        while True:
        # yield MV, wait for new t, SP, PV
            t, PV, SP, TR = yield MV
        
        #adjust I term so output matches tracking input
            I = TR - MV_bar - P - D
        
        # PID calculations
            P = Kp * (beta * SP - PV)
            I = I + Ki*(SP - PV)*(t - t_prev)
            eD = gamma*SP - PV
            D = Kd*(eD - eD_prev)/(t - t_prev)
            MV = MV_bar + P + I + D
        
        # update stored data for next iteration
            eD_prev = eD
            t_prev = t

        
    def record_T(self):
        self.df.loc[self.index,'Temperature'] = lab.T1
        self.Pyro_temp_readout_display_number.setText("{:.1f}".format(self.df.loc[self.index,'Temperature']))
        self.curve_T.setData(self.df['Temperature'].tolist())
        
    def record_I(self):
        self.df.loc[self.index,'Current'] = np.sqrt(lab.U1/self.resistance)
        self.PS_current_readout_display_number.setText("{:.2f}".format(self.df.loc[self.index,'Current'])) 
   
    def record_V(self):
        self.df.loc[self.index,'Voltage'] = np.sqrt(lab.U1*self.resistance)
        self.PS_voltage_readout_display_number.setText("{:.2f}".format(self.df.loc[self.index,'Voltage']))
               
    def record_P(self):
        self.df.loc[self.index,'Power'] = lab.U1 
        self.curve_P.setData(self.df['Power'].tolist())
#TIMER  

    def record_press(self):
        self.df.loc[self.index,'Pressure'] = float(self.Pyro_pres_readout_display_number.text())#random.random()
        self.Pyro_pres_readout_display_number.setText("{:.2e}".format(self.df.loc[self.index,'Pressure']))
         
    def read_Data(self):
        self.df = pd.DataFrame(columns = ['Temperature','Power','Current','Voltage','Pressure'])
        self.df.to_csv('Anneal_Data.csv', index = False)    
        self.curve_T = self.plot_T.plot(pen='y')
        self.curve_P = self.plot_P.plot(pen='y')    
        self.startBtn.clicked.connect(self.record_All)
        self.startBtn.clicked.connect(self.startTimer)
        self.stopBtn.clicked.connect(self.endTimer)   
    
    def make_row(self):
        self.index = self.df.shape[0]
        
    def record_All(self):
        self.timer.timeout.connect(self.make_row)
        self.timer.timeout.connect(self.record_T)
        
        self.timer.timeout.connect(self.record_P)
        self.timer.timeout.connect(self.record_press)
        self.timer.timeout.connect(self.record_V)
        self.timer.timeout.connect(self.record_I)
        self.timer.timeout.connect(self.record_file)

    t = 0
    t_PID = 0
    resistance = 20
    def Anneal(self):
        self.RampBtn.clicked.connect(self.Anneal_control)
        self.RampBtn.clicked.connect(self.startAnneal)
        self.StopAnnealBtn.clicked.connect(self.Stop_Anneal)
        self.StopAnnealBtn.clicked.connect(self.endAnneal) 
    
    def Stop_Anneal(self):
        self.timer.timeout.disconnect(self.Anneal_step)
        self.timer.timeout.connect(self.Safe_Stop_Anneal)
            
    def Stop_Now_Anneal(self):
        self.RampDescr.setText("NO OPERATION") 
        #self.t = 0
        self.t_PID = 0
        self.flag = 0
        lab.U1 = 0
        self.current_setpoint = self.RT_setpoint
        self.timer.timeout.disconnect(self.Safe_Stop_Anneal)
        #self.timer.timeout.disconnect(self.counter)
        self.Target_temp_display.setReadOnly(False)
        self.Ramp_time_display.setReadOnly(False)
        self.Anneal_time_display.setReadOnly(False)
        self.Cooldown_time_display.setReadOnly(False)
        self.Wait_time_display.setReadOnly(False)
    
    def Safe_Stop_Anneal(self):
        self.RampDescr.setText('SAFE SWITCH OFF')
        self.RampStp.setText('stp:' + "{:.1f}".format(self.current_setpoint))
        if self.current_setpoint > self.RT_setpoint & lab.U1>0 :
            self.current_setpoint = self.current_setpoint - 1. 
            PV = lab.T1                                                # get measurement
            MV = self.pid.send([self.t, PV, self.current_setpoint, lab.U1])# compute manipulated variable
            if MV < self.power_limit:
                lab.U1 = MV
            else: lab.U1 = self.power_limit
        else:
            self.Stop_Now_Anneal()
            
    def ramp_to_100(self):
        if self.power_limit>lab.U1:#self.current*self.voltage:
            """
            new_curr = self.current + 0.007 #this correspond to 0.001 on the power supply display
            print(new_curr)
            message = 'ISET1:' + str(new_curr)
            """
            lab.U1 = lab.U1 + 0.5
            #self.pow_sup.write(message)
            PV = lab.T1#self.Temp 
            self.current_setpoint = lab.T1
            MV = self.pid.send([self.t, PV, PV, lab.U1])
        
    def Anneal_step(self):
            self.t_PID = self.t_PID +1 
            current_power = lab.U1
            operations = ['RAMP to 30C','RAMP_PID','ANNEAL','COOLDOWN','WAIT','NO OPERATION']
            current_operation = operations[0]

            if self.flag == 0:
                if float(self.Pyro_temp_readout_display_number.text()) <30:  
                    self.ramp_to_100()
                else:
                    self.step_up = (self.setpoint - self.current_setpoint)/self.ramp_time
                    self.flag = 1        
            else:             
                """
                self.power_limit = int(self.Power_limit_display.text())
                if self.press_limit < float(self.Pyro_pres_readout_display_number.text()):
                    self.power_limit = 0.75*self.power_limit
                """
                if self.ramp_time>0:
                    self.current_setpoint = self.current_setpoint + self.step_up              # get setpoint
                    current_operation = operations[1]
                    self.ramp_time = self.ramp_time - 1
                elif self.anneal_time>0:  
                    current_operation = operations[2]                    # get setpoint
                    self.current_setpoint = self.setpoint
                    self.anneal_time = self.anneal_time - 1
                elif self.cooldown_time > 0:
                    current_operation = operations[3] 
                    self.current_setpoint = self.current_setpoint - self.step_down 
                    self.cooldown_time = self.cooldown_time - 1
                elif self.wait_time > 0:
                    current_operation = operations[4] 
                    lab.U1 = 0
                    self.wait_time = self.wait_time - 1
                else:
                    current_operation = operations[5] 
                    self.current_setpoint = self.RT_setpoint
                    self.Stop_Anneal()
                    self.endAnneal()
                PV = lab.T1                                                # get measurement
                MV = self.pid.send([self.t_PID, PV, self.current_setpoint,lab.U1])# compute manipulated variable
                    #current_power = lab.U1
                if MV < self.power_limit:
                    lab.U1 = MV 
                else: lab.U1 = self.power_limit
            self.RampDescr.setText(current_operation)
            self.RampStp.setText('stp:' + "{:.1f}".format(self.current_setpoint))
    """        
    def counter(self):
        self.t = self.t + 1
        self.setpoint = int(self.Target_temp_display.text())      
        self.ramp_time = int(self.Ramp_time_display.text()) * 60
        self.anneal_time = int(self.Anneal_time_display.text()) * 60
        self.cooldown_time = int(self.Cooldown_time_display.text()) * 60
        self.wait_time = int(self.Wait_time_display.text()) * 60
        self.step_up = (self.setpoint - self.RT_setpoint)/self.ramp_time
        self.step_down = (self.setpoint - self.RT_setpoint)/self.cooldown_time
    """    
    def Create_PID(self):
        P = 1.4
        I = 1
        D = 0.001
        self.pid = self.PID(P,I,D)
        self.pid.send(None)
    
    def time_counter(self):
        self.t = self.t + 1
        
    def Anneal_control(self):   
        self.setpoint = int(self.Target_temp_display.text())      
        self.ramp_time = int(self.Ramp_time_display.text()) * 60
        self.anneal_time = int(self.Anneal_time_display.text()) * 60
        self.cooldown_time = int(self.Cooldown_time_display.text()) * 60
        self.wait_time = int(self.Wait_time_display.text()) * 60
        #self.step_up = (self.setpoint - self.RT_setpoint)/self.ramp_time
        self.step_down = (self.setpoint - self.RT_setpoint)/self.cooldown_time
        self.press_limit = float(self.Press_limit_display.text()) 
        self.power_limit = float(self.Power_limit_display.text())
        self.flag = 0 
        #self.current_setpoint = self.RT_setpoint
        self.Target_temp_display.setReadOnly(True)
        self.Ramp_time_display.setReadOnly(True)
        self.Anneal_time_display.setReadOnly(True)
        self.Cooldown_time_display.setReadOnly(True)
        self.Wait_time_display.setReadOnly(True)
        #self.timer.timeout.connect(self.counter)       
        self.timer.timeout.connect(self.Anneal_step)
                  
    def startTimer(self):
        self.timer.start(1000)
        self.timer.timeout.connect(self.time_counter)
        self.startBtn.setEnabled(False)
        self.stopBtn.setEnabled(True)

    def endTimer(self):
        self.timer.stop()
        self.timer.timeout.disconnect(self.make_row)
        self.timer.timeout.disconnect(self.record_T)
        self.timer.timeout.disconnect(self.record_P)
        self.timer.timeout.disconnect(self.record_press)
        self.timer.timeout.disconnect(self.record_V)
        self.timer.timeout.disconnect(self.record_I)
        self.timer.timeout.disconnect(self.record_file)
        self.startBtn.setEnabled(True)
        self.stopBtn.setEnabled(False)
 
    def startAnneal(self):
        self.RampBtn.setEnabled(False)
        self.StopAnnealBtn.setEnabled(True)

    def endAnneal(self):
        self.RampBtn.setEnabled(True)
        self.StopAnnealBtn.setEnabled(False)
        
if __name__ == "__main__":   
    TCLab = setup(connected=False, speedup=10)
    with TCLab() as lab:
        app = QApplication(sys.argv)
        app.setStyle('Fusion')
        window = Window()
        window.show()
        sys.exit(app.exec_())        
        

TCLab version 0.4.9
Simulated TCLab
TCLab Model disconnected successfully.


SystemExit: 0

C:\Users\mocha\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3405: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:

import pyvisa as visa
rm = visa.ResourceManager()
rm.list_resources('?*')

()

In [ ]:
import pyqtgraph.examples
pyqtgraph.examples.run()